In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Load Datasets

In [ ]:
train_set_path = '/kaggle/input/bike-sharing-demand/train.csv'
test_set_path = '/kaggle/input/bike-sharing-demand/test.csv'

train_df = pd.read_csv(train_set_path)
test_df = pd.read_csv(test_set_path)

In [ ]:
train_df.head()

In [ ]:
test_df.head()

# EDA

In [ ]:
train_df.describe()

In [ ]:
train_df['count'].hist()

In [ ]:
sns.pairplot(train_df[['weather', 'workingday', 'holiday', 'season', 'temp', 'atemp', 'humidity', 'windspeed', 'count']])

In [ ]:
sns.distplot(train_df['count'])

In [ ]:
y_train = np.log(train_df['count'] + 1)

In [ ]:
sns.distplot(y_train)

In [ ]:
train_df.info()

# Feature Engineering

In [ ]:
def create_date_columns(df, col_name):
    df['datetime'] = pd.to_datetime(df[col_name])
    df['hour'] =  df[col_name].apply(lambda date: date.hour)
    df['day'] =  df[col_name].apply(lambda date: date.day)
    df['month'] =  df[col_name].apply(lambda date: date.month)
    df['year'] =  df[col_name].apply(lambda date: date.year)
    df['day_number'] =  df[col_name].apply(lambda date: date.weekday())
    return df

In [ ]:
train_df = create_date_columns(train_df, 'datetime')

In [ ]:
train_df.head()

In [ ]:
sns.barplot(data=train_df, x='day_number', y = 'count')

In [ ]:
sns.barplot(data=train_df, x='hour', y = 'count')

In [ ]:
sns.barplot(data=train_df, x='month', y = 'count')

In [ ]:
sns.boxplot(x=train_df["hour"], y=train_df['count'])

In [ ]:
count_per_hour = train_df.groupby('hour').agg('sum')['count']
count_per_hour

In [ ]:
sns.relplot(data = count_per_hour, kind='line')

In [ ]:
selected_features = ['season', 'holiday', 'workingday', 'weather', 'temp', 'atemp', 'windspeed', 'humidity', 'hour', 'day_number']

# Pipline

In [ ]:
def pipline(df, selected_features):
    df = create_date_columns(df, 'datetime')
    return df[selected_features]

In [ ]:
X_train = pipline(train_df, selected_features)

In [ ]:
X_train.head()

# Split Dataset for training

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2)

# Evaluation Matrix

In [ ]:
def rmsle(real, pred):
    return np.sqrt(np.mean(((np.log(pred + 1)) - (np.log(real + 1)))**2))

# Model selection

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor() # randomforest with default prameters

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_hat_train = model.predict(X_train)
y_hat_valid = model.predict(X_valid)

In [ ]:
print(len(y_hat_train))
print(len(y_hat_valid))

In [ ]:
print(f"train_score: {rmsle(np.exp(y_train)-1, np.exp(y_hat_train) - 1)}")
print(f"validation_score: {rmsle(np.exp(y_valid)-1, np.exp(y_hat_valid) - 1)}")

In [ ]:
model = RandomForestRegressor(n_estimators=500, max_depth=15) 
model.fit(X_train, y_train)

In [ ]:
y_hat_train = model.predict(X_train)
y_hat_valid = model.predict(X_valid)

In [ ]:
print(f"train_score: {rmsle(np.exp(y_train)-1, np.exp(y_hat_train) - 1)}")
print(f"validation_score: {rmsle(np.exp(y_valid)-1, np.exp(y_hat_valid) - 1)}")

# Best Fit model

In [ ]:
best_model = RandomForestRegressor(n_estimators=500, max_depth=15)
X_train = pipline(train_df, selected_features)
y_train = np.log(train_df['count'] + 1)
best_model.fit(X_train, y_train)

In [ ]:
X_test = pipline(test_df, selected_features)

In [ ]:
y_hat = best_model.predict(X_test)

# Submit the answer

In [ ]:
test_df['count'] = np.exp(y_hat) -1
final_df = test_df[['datetime', 'count']].copy()
final_df.to_csv('submission.csv', index=False)

# END